In [ ]:
import tensorflow as tf

2023-09-05 14:57:22.974175: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-05 14:57:23.036500: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-05 14:57:23.037851: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-05 14:57:24.027340: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Define Hyper-Parameters

In [ ]:
PREPROCESSING_ARGS = {
    'sampling_rate': 16000,
    'frame_length_in_s': 0.04,
    'frame_step_in_s': 0.02,
    'num_mel_bins': 40,
    'lower_frequency': 20,
    'upper_frequency': 4000,
}

TRAINING_ARGS = {
    'batch_size': 20,
    'initial_learning_rate': 0.01,
    'end_learning_rate': 1.e-5,
    'epochs': 10
}

# Create Train/Val/Test Datasets

In [ ]:
train_ds = tf.data.Dataset.list_files('msc-train/*')
val_ds = tf.data.Dataset.list_files('msc-val/*')
test_ds = tf.data.Dataset.list_files('msc-test/*')

2022-11-11 14:42:11.267884: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-11 14:42:11.267927: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-11 14:42:11.267949: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-bdbc19a1-4d38-4250-8aec-b2a48a575b76): /proc/driver/nvidia/version does not exist
2022-11-11 14:42:11.268375: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-11 14:42:11.806144: W tensorflow_io/core/kernels/audio_v

TensorShape([49, 321])

In [ ]:
from preprocessing import LABELS
from preprocessing import AudioReader
from preprocessing import MelSpectrogram


audio_reader = AudioReader(tf.int16, 16000)
mel_spec_processor = MelSpectrogram(**PREPROCESSING_ARGS)

def prepare_for_training(feature, label):
    feature = tf.expand_dims(feature, -1)
    label_id = tf.argmax(label == LABELS)

    return feature, label_id


batch_size = TRAINING_ARGS['batch_size']
epochs = TRAINING_ARGS['epochs']

train_ds = (train_ds
            .map(audio_reader.get_audio_and_label)
            .map(mel_spec_processor.get_mel_spec_and_label)
            .map(prepare_for_training)
            .batch(batch_size)
            .cache())
val_ds = (val_ds
            .map(audio_reader.get_audio_and_label)
            .map(mel_spec_processor.get_mel_spec_and_label)
            .map(prepare_for_training)
            .batch(batch_size))
test_ds = (test_ds
            .map(audio_reader.get_audio_and_label)
            .map(mel_spec_processor.get_mel_spec_and_label)
            .map(prepare_for_training)
            .batch(batch_size))

2023-09-05 14:57:25.908834: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available
2023-09-05 14:57:25.908988: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX AVX2 AVX512F FMA


In [ ]:
for example_batch, example_labels in train_ds.take(1):
  print('Batch Shape:', example_batch.shape)
  print('Data Shape:', example_batch.shape[1:])
  print('Labels:', example_labels)

Batch Shape: (20, 49, 40, 1)
Data Shape: (49, 40, 1)
Labels: tf.Tensor([1 0 3 1 4 4 0 1 3 3 1 6 1 2 0 7 2 2 4 3], shape=(20,), dtype=int64)
2023-09-05 14:57:27.166214: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


# Create the Model (DS-CNN)

DS-CNN
Conv2D(filters=256, kernel_size=[3, 3], stride=[2, 2],                   
       use_bias=False, padding=’valid’)
BatchNormalization()
ReLU()
DepthwiseConv2D(kernel_size=[3, 3], stride=[1, 1], 
                use_bias=False, padding=’same’)
Conv2D(filters=256, kernel_size=[1, 1], stride=[1, 1],   
       use_bias=False)
BatchNormalization()
ReLU()
DepthwiseConv2D(kernel_size=[3, 3], stride=[1, 1],
                use_bias=False, padding=’same’)
Conv2D(filters=256, kernel_size=[1, 1], stride=[1, 1],   
       use_bias=False)
BatchNormalization()
ReLU()
GlobalAveragePooling2D()
Dense(units=8)
Softmax()


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=example_batch.shape[1:]),
    tf.keras.layers.Conv2D(filters=256, kernel_size=[3, 3], strides=[2, 2],
        use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1], 
        use_bias=False, padding='same'),
    tf.keras.layers.Conv2D(filters=256, kernel_size=[1, 1], strides=[1, 1],   
       use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.Conv2D(filters=256, kernel_size=[1, 1], strides=[1, 1],   
       use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=len(LABELS)),
    tf.keras.layers.Softmax()
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 19, 256)       2304      
                                                                 
 batch_normalization (Batch  (None, 24, 19, 256)       1024      
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 24, 19, 256)       0         
                                                                 
 depthwise_conv2d (Depthwis  (None, 24, 19, 256)       2304      
 eConv2D)                                                        
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 19, 256)       65536     
                                                                 
 batch_normalization_1 (Bat  (None, 24, 19, 256)       1

# Train the Model

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
initial_learning_rate = TRAINING_ARGS['initial_learning_rate']
end_learning_rate = TRAINING_ARGS['end_learning_rate']

linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=initial_learning_rate,
    end_learning_rate=end_learning_rate,
    decay_steps=len(train_ds) * epochs,
)
optimizer = tf.optimizers.Adam(learning_rate=linear_decay)
metrics = [tf.metrics.SparseCategoricalAccuracy()]
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

history = model.fit(train_ds, epochs=epochs, validation_data=val_ds)

Epoch 1/10
320/320 [==============================] - 215s 667ms/step - loss: 2.0450 - sparse_categorical_accuracy: 0.1989 - val_loss: 1.9776 - val_sparse_categorical_accuracy: 0.2400
Epoch 2/10
320/320 [==============================] - 122s 383ms/step - loss: 1.5691 - sparse_categorical_accuracy: 0.4047 - val_loss: 1.4726 - val_sparse_categorical_accuracy: 0.4625
Epoch 3/10
320/320 [==============================] - 123s 385ms/step - loss: 1.0453 - sparse_categorical_accuracy: 0.6308 - val_loss: 0.9290 - val_sparse_categorical_accuracy: 0.6787
Epoch 4/10
320/320 [==============================] - 132s 412ms/step - loss: 0.7722 - sparse_categorical_accuracy: 0.7362 - val_loss: 0.9221 - val_sparse_categorical_accuracy: 0.6650
Epoch 5/10
320/320 [==============================] - 123s 383ms/step - loss: 0.6143 - sparse_categorical_accuracy: 0.7952 - val_loss: 0.6671 - val_sparse_categorical_accuracy: 0.7700
Epoch 6/10
320/320 [==============================] - 122s 381ms/step - loss: 0.

In [ ]:
history.history

{'loss': [2.0450072288513184,
  1.5691094398498535,
  1.0453083515167236,
  0.7722204327583313,
  0.6142938137054443,
  0.5076009631156921,
  0.4299235939979553,
  0.3681630790233612,
  0.31604674458503723,
  0.27835968136787415],
 'sparse_categorical_accuracy': [0.19890624284744263,
  0.4046874940395355,
  0.6307812333106995,
  0.7362499833106995,
  0.7951562404632568,
  0.8306249976158142,
  0.8595312237739563,
  0.8831250071525574,
  0.903124988079071,
  0.9175000190734863],
 'val_loss': [1.9776300191879272,
  1.4726146459579468,
  0.9289615750312805,
  0.9221228957176208,
  0.667147159576416,
  0.5383950471878052,
  0.48947229981422424,
  0.4217441976070404,
  0.3928365707397461,
  0.3514421582221985],
 'val_sparse_categorical_accuracy': [0.23999999463558197,
  0.4625000059604645,
  0.6787499785423279,
  0.6650000214576721,
  0.7699999809265137,
  0.8025000095367432,
  0.8324999809265137,
  0.8650000095367432,
  0.8725000023841858,
  0.9012500047683716]}

# Test the model

In [ ]:
test_loss, test_accuracy = model.evaluate(test_ds)

40/40 [==============================] - 14s 346ms/step - loss: 0.3192 - sparse_categorical_accuracy: 0.9038


In [ ]:
training_loss = history.history['loss'][-1]
training_accuracy = history.history['sparse_categorical_accuracy'][-1]
val_loss = history.history['val_loss'][-1]
val_accuracy = history.history['val_sparse_categorical_accuracy'][-1]

print(f'Training Loss: {training_loss:.4f}')
print(f'Training Accuracy: {training_accuracy*100.:.2f}%')
print()
print(f'Validation Loss: {val_loss:.4f}')
print(f'Validation Accuracy: {val_accuracy*100.:.2f}%')
print()
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy*100.:.2f}%')

Training Loss: 0.2784
Training Accuracy: 91.75%

Validation Loss: 0.3514
Validation Accuracy: 90.13%

Test Loss: 0.3192
Test Accuracy: 90.38%


# Save the Model

### Save Keras Model

In [ ]:
import os
from time import time

timestamp = int(time())

saved_model_dir = f'./saved_models/{timestamp}'
if not os.path.exists(saved_model_dir):
    os.makedirs(saved_model_dir)
model.save(saved_model_dir)

INFO:tensorflow:Assets written to: ./saved_models/1693927703/assets
INFO:tensorflow:Assets written to: ./saved_models/1693927703/assets


### Save Hyper-Parameters and Results

In [ ]:
import pandas as pd

output_dict = {
    'timestamp': timestamp,
    **PREPROCESSING_ARGS,
    **TRAINING_ARGS,
    'test_accuracy': test_accuracy
}

df = pd.DataFrame([output_dict])

output_path='./mel_spectrogram_dscnn_results.csv'
df.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6f1fd91f-a434-4542-983d-3ce5ae14ac33' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>